In [3]:
import os
import glob
import pandas as pd
import json
from dotenv import load_dotenv
from functools import partial
from db_tools import get_db

In [4]:
model = "bruss"
run_id = "dist"


In [9]:
load_dotenv()
data_dir = os.getenv("DATA_DIR")
output_dir = os.getenv("OUT_DIR")
os.makedirs(output_dir, exist_ok=True)

df0 = get_db(data_dir, model)
df = df0[df0.run_id == run_id]

In [15]:
df.sigma_ic_u.unique()

array([0.05, 0.01, 0.5 , 0.1 ])